# Manipulación de datos faltantes

Es común en aplicaciones reales que nuestros datos tengan uno o más valores faltantes por diversos motivo: un error en la lectura, una pregunta no respondida en un encuesta, etc. Lo que se observa en los datos es simplemente un faltante en esa ubicación _(NaN, None, Null)._

Desafortunadamente muchos algortimos no possen la capacidad de manipular dichos datos, incluso podrain producir resultados impredecibles si se ignoran: se deben tomar en cuenta esos valores faltantes antes de proceder con el análisis.

Si nosotros sólo le ponemos $0$ a esos valores, lo que estamos haciendo realmente es sesgando nuestro muestra, provocando que haya errores en nuestro modelo.

In [2]:
import numpy as np
import pandas as pd
from io import StringIO

In [3]:
datos = \
"""A,B,C,D
1.0,2.1,3.7,4.5
5.9,6.2,,8.6
9.3,0.4,1.8,"""
df = pd.read_csv(StringIO(datos))
df

,A,B,C,D
0,1.0,2.1,3.7,4.5
1,5.9,6.2,NaN,8.6
2,9.3,0.4,1.8,NaN


En este ejemplo podemos ver que existe valores faltantes en Dataframe de pandas.
Podemos utilizar em metodo `df.isnull()` para indicar si una celda contiene un valor numeric y con `sum()` se obtiene el numero fantantes por columna

In [4]:
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [5]:
# Conociendo el total de valores nulos
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

## Eliminando valores faltantes
Una de las manera más simple de manipular valores faltantes es eliminar **caractetisticas**(columnas, feactures) o **muestras**(filas) que contenga valores faltantes.

In [7]:
# Para eliminar las filas
df.dropna(axis=0)

,A,B,C,D
0,1.0,2.1,3.7,4.5


In [8]:
df

,A,B,C,D
0,1.0,2.1,3.7,4.5
1,5.9,6.2,NaN,8.6
2,9.3,0.4,1.8,NaN


In [9]:
# Para las columnas, deshace las columnas que tenga NaN
df.dropna(axis=1)

,A,B
0,1.0,2.1
1,5.9,6.2
2,9.3,0.4


No es tan recomendable hacer eso, porque estamos tirando el 50% de nuestros datos, y puede ser columnar más importantes para nuestros algortimos, debemos de pensarlo muy bien.

In [6]:
# Elimna fila que tengas todas NaN (primera opcion en pensar)
df.dropna(how='all')

,A,B,C,D
0,1.0,2.1,3.7,4.5
1,5.9,6.2,NaN,8.6
2,9.3,0.4,1.8,NaN


dropna tiene mas parametros, ver la documentacion para experimentar.

In [8]:
# Elimina solo la fila que tena un NAN
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.1,3.7,4.5
2,9.3,0.4,1.8,NaN


## Sustituyendo valores faltantes
En ocasiones noes recomendable eliminar caracteristicas o muestras compleatas debido a que se podria perder dsatos valioso en el procesos; en este caso se puede utilizar **tecnicas de interpolación** para estimar los valores faltantes a partir de otras muestras en nuestro conjunto de datos, 

Una usadad es la **media por características** (columnas).

In [31]:
datos_n = df.copy() # No afectar al df original
for col in df.columns.values:
    # Si es cero es que devolvio todas False 
    falta = np.sum(df[col].isnull())
    if falta:
        print('Asignado {} valores en columna: {}'.format(falta, col))
        # Sacamos el promedio de la columna
        mean = df[col].mean()
        # Llena cada valor NA/NULL con el promedio de la columna
        datos_n[col] = df[col].fillna(mean)

# Dataframe devuelvo por fillna
datos_n        

Asignado 1 valores en columna: C
Asignado 1 valores en columna: D


,A,B,C,D
0,1.0,2.1,3.70,4.50
1,5.9,6.2,2.75,8.60
2,9.3,0.4,1.80,6.55


In [33]:
df

,A,B,C,D
0,1.0,2.1,3.7,4.5
1,5.9,6.2,NaN,8.6
2,9.3,0.4,1.8,NaN


Esto fue por columna, tambien puede ser por fila.

In [9]:
# Tarea moral
# Code here para hacerlo ahora con fila.

In [21]:
df.columns.values

array(['A', 'B', 'C', 'D'], dtype=object)

In [22]:
df['C']

0    NaN
1    NaN
2    1.8
Name: C, dtype: float64

In [23]:
df['C'].mean()

1.8

In [24]:
df['C'].fillna(555)

0    555.0
1    555.0
2      1.8
Name: C, dtype: float64

Se puede utilizar la clase `SimpleImputer` para sustituir los datos faltantes:

In [35]:
df.values

array([[1. , 2.1, 3.7, 4.5],
       [5.9, 6.2, nan, 8.6],
       [9.3, 0.4, 1.8, nan]])

### Implementacion con sklearn

In [39]:
from sklearn.impute import SimpleImputer

# Creacion del objeto con requerimientos que deseamos
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# Calula el strategy para cada columna.
imp.fit(df.values)
# Valores guardados como atributo del objeto.
# print(imp.statistics_)
# Completa los valos NA con los valores de imp.statistics
datos_n = imp.transform(df.values)
# Devuleve un data frame con los valores ya llenados
datos_n

array([[1.  , 2.1 , 3.7 , 4.5 ],
       [5.9 , 6.2 , 2.75, 8.6 ],
       [9.3 , 0.4 , 1.8 , 6.55]])

    impute
    Most machine learning algorithms require that their inputs have no missing values, and will not work if this requirement is violated. Algorithms that attempt to fill in (or impute) missing values are referred to as imputation algorithms.


    When you run imp.fit, it calculates the value to replace in each column (in this case the mean).
    You can access access what I keep in each column with: imp.statistics_

    When you run imp.transform, it complete the NaN with the corresponding value (what is in imp.statistics_)

- https://scikit-learn.org/stable/modules/impute.html
- https://stackoverflow.com/questions/52416187/what-does-imputer-fit-with-nan-value-do
- https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer


**Nota**.Esto solo lo hace por columnas, tiene sentido por ser que cada columna es una conjunto de caracteristicas del mismo tipo.

Otros opciones para el parámetro strategy son `medina` y `most_frequent` coloca el valor más repetido en lugar de los faltantes y es muy util cuando se tienen **datos categóricos:**

In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame([["a", np.nan],
                   ["a", "y"],
                   ["b", "y"],
                   ["c"], ["x"], 
                   [np.nan], ["z"]],
                 dtype="category")
df

,0,1
0,a,NaN
1,a,y
2,b,y
3,c,NaN
4,x,NaN
5,NaN,NaN
6,z,NaN


In [13]:
imp = SimpleImputer(strategy="most_frequent") # la moda
print(imp.fit_transform(df))

[['a' 'y']
 ['a' 'y']
 ['b' 'y']
 ['c' 'y']
 ['x' 'y']
 ['a' 'y']
 ['z' 'y']]
